In [1]:
!pip install openfisca-us

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 25.9 MB/s 
     |████████████████████████████████| 96 kB 5.6 MB/s 
     |████████████████████████████████| 208 kB 44.6 MB/s 
     |████████████████████████████████| 248 kB 42.6 MB/s 
     |████████████████████████████████| 15.3 MB 509 kB/s 
     |████████████████████████████████| 79 kB 7.1 MB/s 
INFO: pip is looking at multiple versions of openfisca-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of openfisca-core[web-api] to determine which version is compatible with other requirements. This could take a while.
     |████████████████████████████████| 205 kB 48.5 MB/s 
     |████████████████████████████████| 207 kB 49.3 MB/s 
     |████████████████████████████████| 208 kB 13.8 MB/s 
     |████████████████████████████████| 208 kB 45.1 MB/s 
     |████████

In [17]:
from openfisca_us import IndividualSim
from plotly import express as px
import pandas as pd
import numpy as np

In [3]:
def get_threshold(adults, children):
    # TODO: Map from filing status instead to match the law.
    if adults == 1 and children == 0:
        return 125_000
    return 250_000


def get_dead_zone(cliff, adults, children, state_code):
    # Create simulation.
    threshold = get_threshold(adults, children)
    sim = IndividualSim(year=2021)
    members = ["head"]
    sim.add_person(name="head", age=25)
    if adults == 2:
        spouse = "spouse"
        members.append(spouse)
        sim.add_person(name=spouse, age=25)
    for i in range(children):
        child = "child" + str(i)
        sim.add_person(name=child, age=10)
        members.append(child)
    sim.add_tax_unit(members=members)
    sim.add_spm_unit(members=members)
    sim.add_household(members=members, state_code=state_code)
    sim.vary("employment_income", min=threshold, max=(threshold + 2 * cliff), step=10)
    # Calculate earnings dead zone.
    df = pd.DataFrame(dict(
        employment_income = sim.calc("employment_income")[0],
        net_income = sim.calc("spm_unit_net_income")[0]
    ))
    return df[df.net_income >= (df.net_income[0] + cliff)].iloc[0].employment_income - threshold

In [4]:
# Test with one example.
get_dead_zone(10_000, 1, 0, "CA")

14640.0

In [5]:
l = []
for cliff in [10_000, 20_000]:
    for adults in [1, 2]:
        for children in [0, 1, 2]:
            for state_code in ["MA", "MD", "WA"]:
                dead_zone = get_dead_zone(cliff, adults, children, state_code)
                tmp = pd.Series(dict(
                    cliff=cliff, adults=adults, children=children,
                    state_code=state_code, dead_zone=dead_zone
                ))
                print(tmp)
                l.append(tmp)
df = pd.concat(l, axis=1).T

cliff           10000
adults              1
children            0
state_code         MA
dead_zone     15790.0
dtype: object
cliff           10000
adults              1
children            0
state_code         MD
dead_zone     15900.0
dtype: object
cliff           10000
adults              1
children            0
state_code         WA
dead_zone     14640.0
dtype: object
cliff           10000
adults              1
children            1
state_code         MA
dead_zone     17350.0
dtype: object
cliff           10000
adults              1
children            1
state_code         MD
dead_zone     17500.0
dtype: object
cliff           10000
adults              1
children            1
state_code         WA
dead_zone     15970.0
dtype: object
cliff           10000
adults              1
children            2
state_code         MA
dead_zone     19000.0
dtype: object
cliff           10000
adults              1
children            2
state_code         MD
dead_zone     19180.0
dtype: object
cliff   

Example: MA single adult no kids
1. 125000 (transparent)
2. 15790 (10000 cliff)
3. 18910 (difference between 20000 cliff of 34,700 and 10000 cliff)

In [6]:
df["threshold"] = df.apply(lambda row: get_threshold(row.adults, row.children), axis=1)

In [7]:
# May need to use melt and/or pivot to get the data into the right format.
df

,cliff,adults,children,state_code,dead_zone,threshold
0,10000,1,0,MA,15790.0,125000
1,10000,1,0,MD,15900.0,125000
2,10000,1,0,WA,14640.0,125000
3,10000,1,1,MA,17350.0,250000
4,10000,1,1,MD,17500.0,250000
5,10000,1,1,WA,15970.0,250000
6,10000,1,2,MA,19000.0,250000
7,10000,1,2,MD,19180.0,250000
8,10000,1,2,WA,17350.0,250000
9,10000,2,0,MA,14570.0,250000


TODO:
* Loop through
  * 10k/20k
  * Married/non-married
  * Number of children: 0/1/2
  * State: MD/MA/WA
* Graph:
  * Combined bars with extra for Pell grant
  * 6 bars for married/non and 0/1/2 children
  * Slider (animation frame) for US state

See https://medium.com/@moritzkoerber/how-to-plot-a-grouped-stacked-bar-chart-in-plotly-df1685b83460

In [8]:
df_MA = df.loc[df['state_code']=='MA']
df_MA

,cliff,adults,children,state_code,dead_zone,threshold
0,10000,1,0,MA,15790.0,125000
3,10000,1,1,MA,17350.0,250000
6,10000,1,2,MA,19000.0,250000
9,10000,2,0,MA,14570.0,250000
12,10000,2,1,MA,14570.0,250000
15,10000,2,2,MA,14570.0,250000
18,20000,1,0,MA,30350.0,125000
21,20000,1,1,MA,34700.0,250000
24,20000,1,2,MA,37300.0,250000
27,20000,2,0,MA,29140.0,250000


In [9]:
import plotly.graph_objects as go

In [10]:
cliff = df_MA['cliff'].unique()
cliff
type(cliff[0])

int

In [11]:
df_MA_10000 = df_MA.loc[df_MA['cliff'] == 10000]
df_MA_10000

,cliff,adults,children,state_code,dead_zone,threshold
0,10000,1,0,MA,15790.0,125000
3,10000,1,1,MA,17350.0,250000
6,10000,1,2,MA,19000.0,250000
9,10000,2,0,MA,14570.0,250000
12,10000,2,1,MA,14570.0,250000
15,10000,2,2,MA,14570.0,250000


In [12]:
fig = go.Figure()

fig.update_layout(
    xaxis=dict(title_text="Employment Income"),
    yaxis=dict(title_text="Net Income"),
    barmode="stack"
)

colors = ["#2A66DE", "#FFC32B"]

for r, c in zip(cliff, colors):
    plot_df = df_MA[df.cliff == r]
    fig.add_trace(
        go.Bar(x=[plot_df.adults, plot_df.children], y=plot_df.dead_zone, name=r, marker_color=c),
    )

fig


In [13]:
df_MA_extra_columns = df_MA.copy()
df_MA_extra_columns

,cliff,adults,children,state_code,dead_zone,threshold
0,10000,1,0,MA,15790.0,125000
3,10000,1,1,MA,17350.0,250000
6,10000,1,2,MA,19000.0,250000
9,10000,2,0,MA,14570.0,250000
12,10000,2,1,MA,14570.0,250000
15,10000,2,2,MA,14570.0,250000
18,20000,1,0,MA,30350.0,125000
21,20000,1,1,MA,34700.0,250000
24,20000,1,2,MA,37300.0,250000
27,20000,2,0,MA,29140.0,250000


In [14]:
df_MA_extra_columns['adults_str']= np.where(
    df_MA_extra_columns.adults == 2, "Two adults", "One adult"
)
df_MA_extra_columns['children_str']= np.select(
    [df_MA_extra_columns.children == 1, df_MA_extra_columns.children == 2],
    ["One child", "Two children"], "No children")
df_MA_extra_columns

,cliff,adults,children,state_code,dead_zone,threshold,adults_str,children_str
0,10000,1,0,MA,15790.0,125000,1 adult(s),0 child(ren)
3,10000,1,1,MA,17350.0,250000,1 adult(s),1 child(ren)
6,10000,1,2,MA,19000.0,250000,1 adult(s),2 child(ren)
9,10000,2,0,MA,14570.0,250000,2 adult(s),0 child(ren)
12,10000,2,1,MA,14570.0,250000,2 adult(s),1 child(ren)
15,10000,2,2,MA,14570.0,250000,2 adult(s),2 child(ren)
18,20000,1,0,MA,30350.0,125000,1 adult(s),0 child(ren)
21,20000,1,1,MA,34700.0,250000,1 adult(s),1 child(ren)
24,20000,1,2,MA,37300.0,250000,1 adult(s),2 child(ren)
27,20000,2,0,MA,29140.0,250000,2 adult(s),0 child(ren)


In [15]:
fig = go.Figure()

fig.update_layout(
    xaxis=dict(title_text="Family Size"),
    yaxis=dict(title_text="Dead Zone"),
    barmode="stack"
)

colors = ["#2A66DE", "#FFC32B"]

for r, c in zip(cliff, colors):
    plot_df = df_MA_extra_columns[df.cliff == r]
    fig.add_trace(
        go.Bar(x=[plot_df.adults_str, plot_df.children_str], y=plot_df.dead_zone, name=r, marker_color=c),
    )

fig


In [16]:
fig = go.Figure()

fig.update_layout(
    xaxis=dict(title_text="Dead Zone"),
    yaxis=dict(title_text="Family Size"),
    barmode="stack"
)

colors = ["#2A66DE", "#FFC32B"]

for r, c in zip(cliff, colors):
    plot_df = df_MA_extra_columns[df.cliff == r]
    fig.add_trace(
        go.Bar(y=[plot_df.adults_str, plot_df.children_str], x=plot_df.dead_zone, name=r, marker_color=c, orientation='h'),
    )

fig


In [29]:
df["label"] = np.core.defchararray.add(
    np.where(
    df.adults == 2, "Married, ", "Single, "),
    np.select(
    [df.children == 1, df.children == 2],
    ["one child", "two children"], "no children"))
df

,cliff,adults,children,state_code,dead_zone,threshold,label
0,10000,1,0,MA,15790.0,125000,"Single, no children"
1,10000,1,0,MD,15900.0,125000,"Single, no children"
2,10000,1,0,WA,14640.0,125000,"Single, no children"
3,10000,1,1,MA,17350.0,250000,"Single, one child"
4,10000,1,1,MD,17500.0,250000,"Single, one child"
5,10000,1,1,WA,15970.0,250000,"Single, one child"
6,10000,1,2,MA,19000.0,250000,"Single, two children"
7,10000,1,2,MD,19180.0,250000,"Single, two children"
8,10000,1,2,WA,17350.0,250000,"Single, two children"
9,10000,2,0,MA,14570.0,250000,"Married, no children"


In [34]:
px.bar(df, y="label", x="dead_zone", color="cliff", animation_frame="state_code", orientation="h")

In [46]:
white_zone = df[df.cliff==10000].copy()
white_zone.dead_zone = white_zone.threshold
white_zone.cliff = "Empty"
combined = pd.concat([white_zone, df])
combined
combined["cliff_label"] = np.select(
    [combined.cliff == 10000, combined.cliff == 20000],
    ["No Pell Grant", "Pell Grant"], "Empty")

In [47]:
px.bar(combined, y="label", x="dead_zone", color="cliff_label", animation_frame="state_code", orientation="h")

In [50]:
wide = combined.pivot(["state_code", "label"], "cliff_label", "dead_zone").reset_index()
wide["Pell Grant"] -= wide["No Pell Grant"]
wide

cliff_label,state_code,label,Empty,No Pell Grant,Pell Grant
0,MA,"Married, no children",250000,14570.0,14570.0
1,MA,"Married, one child",250000,14570.0,14570.0
2,MA,"Married, two children",250000,14570.0,14570.0
3,MA,"Single, no children",125000,15790.0,14560.0
4,MA,"Single, one child",250000,17350.0,17350.0
5,MA,"Single, two children",250000,19000.0,18300.0
6,MD,"Married, no children",250000,14680.0,14670.0
7,MD,"Married, one child",250000,14680.0,14670.0
8,MD,"Married, two children",250000,14680.0,14670.0
9,MD,"Single, no children",125000,15900.0,14680.0


In [69]:
LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

molten = wide.melt(["state_code", "label"], ["Empty", "No Pell Grant", "Pell Grant"])
fig = px.bar(molten, y="label", x="value", color="cliff_label", animation_frame="state_code", orientation="h",
       color_discrete_map={"Empty": "#f1f8e9", "No Pell Grant": "black", "Pell Grant": GRAY})
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
    yaxis_title=None
)
fig.show()